# AstraZeneca - Accumulations
**Important:** Total of Count HCP is not exported. Add it manually!

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df_list = tabula.read_pdf("Switzerland_EFPIA Standard Disclosure Summary Report_2019_engl.pdf", pages='all', lattice=True, multiple_tables=True)

## Concat Tables
In der letzten Tabelle sind die Columns verschoben. Manuell zurücksetzen und zu einem DF joinen

In [3]:
df_last = df_list[len(df_list) - 1]

#Remove Columns 8 and 12
df_last = df_last.drop(columns=[8, 12])

#Reset Columns name
df_last.columns = [ i for i in range(0,13)]

df_list[len(df_list) - 1] = df_last

# Concat tables
df = pd.concat(df_list)

## Format Table

In [12]:
df_export = df.copy()

#Drop Columns
df_export = df_export.drop(df.columns[9:],axis=1)
df_export = df_export.drop(df.columns[0],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

#fill na
df_export = df_export.fillna("")
# Select
amount = df_export[df_export.type.str.contains('Transfers of Value')]
recipients = df_export[df_export.type.str.contains('Number of Recipients in Aggregate Disclosure')]
rnd = df_export[df_export.type.str.contains('Research & Development')]

df_export = pd.concat([amount, recipients, rnd])

# Remove RND Aggregated
df_export = df_export[~df_export['type'].str.contains('Aggregate Disclosure - Research & Development')]


#Numberize
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)
#df_export = sum_amounts(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)

df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN

export_acumulations(df_export, 'astrazeneca')

saved


In [7]:
df_export

,type,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,source
0,hcp_amount,0.0,0.0,1118.0,11309.0,21715.0,1562.0,35704.0,astrazeneca
1,hco_amount,0.0,0.0,0.0,NaN,0.0,0.0,NaN,astrazeneca
2,hcp_count,0.0,0.0,3.0,3.0,3.0,2.0,NaN,astrazeneca
3,hco_count,0.0,0.0,0.0,NaN,0.0,0.0,NaN,astrazeneca
4,rnd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,astrazeneca
